## Cal-CRAI metric: # of chill hours
This notebook generates the text metadata files for the extreme heat exposure metric: `change in average number of seasonal chill hours` using Cal-Adapt: Analytics Engine data. Because the AE data represents 200+ GB of data, metrics were calculated with a cluster in a high performance computing environment (i.e. a pcluster). Please see the processing script `climate_ae_chill_hours.py` for full methodological process. 

### Step 1: Generate metadata

In [ ]:
import pandas as pd
import os
import sys

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [ ]:
df_in = pd.read_csv('climate_heat_chill_hours_metric.csv') # make sure this is in the same folder!
df_in # check

In [ ]:
@append_metadata
def extreme_heat_chill_hours_process(df, export=False, export_filename=None, varname=''):
    '''
    Reduces the size of the initial daily raw temperature data in order to streamline compute time.
    Transforms the raw data into the following baseline metrics:
    * change in number of average annual seasonal chill hours
    
    Methods
    -------
    Metric is calculated with the "45°F and under" model for the cold season (NDJF).
    
    Parameters
    ----------
    df: pd.DataFrame
        Input data.
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI extreme heat metric to AWS
        True = will upload resulting df containing CAL CRAI extreme heat metric to AWS
    export_filename: string
        name of csv file to be uploaded to AWS
    varname: string
        Final metric name, for metadata generation
        
    Script
    ------
    Metric calculation: climate_ae_chill_hours.py via pcluster run
    Metadata generation: climate_ae_chill_hours_metadata.ipynb
        
    Note
    ----
    Because the climate projections data is on the order of 2.4 TB in size, intermediary
    processed files are not produced for each stage of the metric calculation. All processing
    occurs in a single complete run in the notebook listed above.
    '''
        
    # calculate chronic with 2°C WL
    print('Data transformation: raw projections data retrieved for warming level of 2.0°C, by manually subsetting based on GWL for parent GCM and calculating 30 year average.')
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")
    print("Data transformation: drop all singleton dimensions (scenario).")
    print("Data transformation: data subsetted for cold season (NDJF).")
    print("Data transformation: number of chill hours calculated by summing the number of hours per year below 45°F threshold.")

    # historical baseline
    print("Data transformation: historical baseline data retrieved for 1981-2010, averaging across models.")
    print("Data transformation: dynamically-downscaled climate data subsetted for a-priori bias-corrected models.")
    print("Data transformation: drop all singleton dimensions (scenario).")
    print("Data transformation: data subsetted for cold season (NDJF).")
    print("Data transformation: number of chill hours calculated by summing the number of hours per year below 45°F threshold.")

    # calculate delta signal
    print("Data transformation: delta signal calculated by taking difference between chronic (2.0°C) and historical baseline.")

    # reprojection to census tracts
    print("Data transformation: data transformed from xarray dataset into pandas dataframe.")
    print("Data transformation: data reprojected from Lambert Conformal Conic CRS to CRS 3857.")
        
    # min-max standardization
    print("Data transformation: data min-max standardized with min_max_standardize function.")
    
    # export data as csv
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [export_filename]
        upload_csv_aws(export_filename, bucket_name, directory)
        
        # # Check if file exists before attempting to remove it
        # if os.path.exists('climate_heat_chill_hours_metric.csv'):
        #     os.remove('climate_heat_chill_hours_metric') # remove from local to clear up directory
        
        # if os.path.exists(export_filename[0]):
        #     os.remove(export_filename[0])
    
    if export == False:
        print(f'{export_filename} uploaded to AWS.')

In [ ]:
extreme_heat_chill_hours_process(df_in, export=False, export_filename ='climate_heat_chill_hours_metric.csv', varname='test') # varname)